# Multilevel regression modeling with CmdStanPy and plotnine

This notebook provides an introduction to multilevel regression modeling
using [Stan](https://mc-stan.org) and the [CmdStanPy](https://mc-stan.org/cmdstanpy/) interface,
a pure Python3 package for Linux, MacOS, and Windows.
It is based on Chris Fonnesbeck's excellent [A Primer on Bayesian Multilevel Modeling using PyStan](https://mc-stan.org/users/documentation/case-studies/radon.html), which was developed as part of a [Stan workshop](https://statmoddev.stat.columbia.edu/2016/06/09/a-primer-on-bayesian-multilevel-modeling-using-pystan/)  for biomedical statisticians at Vanderbilt University.
The data and models are taken from chapter 12 of the book _Data Analysis Using Regression and Multilevel/Hierarchical Models_, by Andrew Gelman and Jennifer Hill, Cambridge university press, 2006.

The second aim of this notebook is to demonstrate best practices of Bayesian data analysis.
It is important to have a clear understanding of both the analysis goals and the sizes,
shapes, and tendancies of the available data before proceeding to model building.
When constructing models, we are faced with a series of engineering decisions with respect
to the generality, performance, and maintainability of the model.
To visualize data and inferences we use
[plotnine](https://plotnine.readthedocs.io/en/stable/),
an implementation of a [_grammar of graphics_](https://vita.had.co.nz/papers/layered-grammar.pdf) in Python,
based on [ggplot2](https://en.wikipedia.org/wiki/Ggplot2).
To manage tabular data in Python we will be using both [numpy](https://numpy.org/doc/stable/) and [pandas](https://pandas.pydata.org/docs/).

In [ ]:
# import all libraries used in this notebook
import os
import numpy as np
import pandas as pd
from cmdstanpy import CmdStanModel

# plotting libs
import matplotlib.pyplot as plt
import plotnine as p9

# combine multiple plots
import patchworklib as pw  # ignore warning about seaborn
from plotnine.data import *

# suppress plotnine warnings
import warnings
warnings.filterwarnings('ignore')

# setup plotnine look and feel
p9.theme_set(
  p9.theme_grey() + 
  p9.theme(text=p9.element_text(size=10),
        plot_title=p9.element_text(size=14),
        axis_title_x=p9.element_text(size=12),
        axis_title_y=p9.element_text(size=12),
        axis_text_x=p9.element_text(size=8),
        axis_text_y=p9.element_text(size=8)
       )
)
xlabels_90 = p9.theme(axis_text_x = p9.element_text(angle=90, hjust=1))

## Modeling goal: home radon measurement


The goal of the radon study is to provide reasonable estimates
of home radon levels in each of the approximately 3000 counties in the United States.
Gelman and Hill write

>Radon is a carcinogen — a naturally occurring radioactive gas whose decay products are also radioactive — known to cause lung cancer in high concentrations and estimated to cause several thousand lung cancer deaths per year in the United States. The distribution of radon levels in U.S. homes varies greatly, with some houses having dangerously high concentrations. In order to identify the areas with high radon exposures, the Environmental Protection Agency coordinated radon measurements in a random sample of more than 80,000 houses throughout the country.

[Radon gas](https://en.wikipedia.org/wiki/Radon) is a product of the slow decay of uranium into lead.  Due to local differences in geology, the level of exposure to radon gas differs from place to place. A common source is uranium-containing minerals in the ground, and therefore it accumulates in subterranean areas such as basements.

![How radon enters the home](img/radon_entry.jpg)
Image from https://www.health.state.mn.us/communities/environment/air/radon/index.html

The [EPA radon map](https://www.epa.gov/sites/default/files/2015-07/documents/zonemapcolor.pdf) shows the counties of the US, color coded by radon level - red for the highest risk, orange for moderate, and yellow for low.

![EPA radon map](img/epa_radon_map.png)

## Study data:  item-level radon, county-level soil uranium levels

The data comes from EPA surveys at the state and national level carried out in the 1990s.
It is available from the Gelman and Hill [ARM website](http://www.stat.columbia.edu/~gelman/arm/examples/radon/), together with the R scripts used to produce the examples in the book.
The study data is in two files.

- Home radon measurements, and the floor (ground or basement) where measured from an EPA survey conducted in 1992.
The survey data is in file http://www.stat.columbia.edu/~gelman/arm/examples/radon/srrs2.dat.

- County level measurements of soil uranium levels in parts per million.
The soil uranium data is in file http://www.stat.columbia.edu/~gelman/arm/examples/radon/cty.dat


We have downloaded and renamed these files.
File `srrs2.dat` is available as  [data/raw_radon.csv](data/raw_radon.csv) and
file `cty.dat` is available as [data/raw_uranium.csv](data/raw_uranium.csv).
These names are informative and will sort together in a directory listing.
The full EPA dataset has been [archived separately](http://www.stat.columbia.edu/~gelman/arm/examples/radon_complete/).

There are a total of 120K home radon measurements from 3000 US counties.
The per-county measurements follow the population density; there are few or no measurements
for sparsely populated counties, i.e. rural counties and correspondingly more for metropolitan counties.

**Data preprocessing required by this analysis**

Our analysis will use both the home radon data measurements and the county level data.
The results are in two files

+ [data/mn_radon.csv](data/mn_radon.csv) contains the individual home radon measurments.
+ [data/mn_counties.csv](data/mn_counties.csv) contains county-level data.

We need to extract and combine that subset of the information in these tables into the dataset required for this analysis.
The essential pre-processing steps are

1. Merge the county-level soil uranium level measurment into the house-level radon data.

2. Put both outcome and predictors on the log scale, following Gelman and Hill, chapter 4, section 12.

3. Restrict the dataset to Minnesota.

4. Get counts of observations per county.

The full set of pre-processing operations are show in Appendix A.

In [ ]:
mn_radon = pd.read_csv(os.path.join('data','mn_radon.csv'))
mn_radon[:7]

In [ ]:
mn_counties = pd.read_csv(os.path.join('data','mn_counties.csv'))
mn_counties[:3]

## Best Practice #1: preliminary data analysis

A sometimes overlooked issue when doing model criticism and model comparison is the fact that we are evaluating the fit of the model _to the data_.  The size and shape of the data informs our choice of model.  Finally, the data collected is not always the data expected.  Therefore we start with plots and summaries of the raw data. When plotting radon measurments against other data elements, we always plot `log_radon` on the y axis, since it is the outcome variable of interest for this case study.

**A brief introduction to plotnine**

A grammar of graphics defines a plot in terms of:

+ a dataset - in plotnine, this is a pandas.DataFrame
+ a set of mappings from dataset variables to graph elements called "aesthetics"
+ one scale for each aesthetic mapping used
+ a coordinate system
+ a facet specification
+ one or more layers, with each layer having
   + a dataset and aestheic mapping - by default, the plot dataset and mappings are used
   + one geometric object ("geoms")
   + one statistical transformation ("stats") 
   + one position adjustment

The power of ggplot2/plotnine is a combination of
a rich set of geometric objects - points, lines, bar charts, maps, and more;
the ability to filter data used by plots, geoms, and stats;
the ability to compose layers.
But there are limits to compositionality;
all layers need to have compatible coordinate systems and scales.
When developing a visualization, the initial choice of geom might
not work properly with additional layers.
Plot building, like model building, is an iterative process.
A good resource is [Data Visualization with Plotnine](https://f0nzie.github.io/rmarkdown-python-plotnine/),
a Python translation of [R for Data Science](https://r4ds.had.co.nz/).

**First questions: amount of data, variable of interest**

As noted as the outset, the measurements per county vary with the population density.  Therefore a logical first question is:  how much data is there for Minnesota?

In [ ]:
print(f'number of houses: {len(mn_radon)}')
print(f'number of counties: {len(mn_counties)}')

The goal of our analysis is to estimate home radon levels; therefore the outcome variable of interest is `log_radon`.
We use the [pandas.DataFrame.describe](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html) function to get summary statistics over the observed outcome `log_radon`.

In [ ]:
print(f'log_radon summary statistics\n{mn_radon["log_radon"].describe()}')

**Relationship between radon and floor**

As the radon pathways diagram shows, radon comes from the soil, therefore the floor level on which the measurement was taken should be a good predictor of the observed radon level. This is coded as "0" for basement and "1" for ground floor level.  Most of the observations in the survey were taken on the basement level.

In [ ]:
pct_1 = round((mn_radon.floor.sum() / len(mn_radon) * 100))
pct_0= round(100 - pct_1)
print(f'floor 0: {pct_0}%\nfloor 1: {pct_1}%')

The [plotnine geom_histogram](https://plotnine.readthedocs.io/en/stable/generated/plotnine.geoms.geom_histogram.html) provides a visualization of the distribution of home radon measurements in the survey dataset.
Here we overlay two per-floor histograms.
They clearly show the different amounts of observation (y axis), but the trend in log radon levels (x axis) is unclear.

In [ ]:
# overlay historgrams
(p9.ggplot(data=mn_radon, mapping=p9.aes(x='log_radon'))
    + p9.geom_histogram(data=(mn_radon.loc[mn_radon['floor']==0]), fill='orange', color='darkgreen', binwidth=0.2, alpha=0.7)
    + p9.geom_histogram(data=(mn_radon.loc[mn_radon['floor']==1]), fill='violet', color='blue', binwidth=0.2, alpha=0.6)
    + p9.xlab("log radon levels") + p9.ylab("number of observations")
    + p9.theme(figure_size=(6,4))
)

Alternatively, the [plotnine geom_density](https://plotnine.readthedocs.io/en/stable/generated/plotnine.geoms.geom_density.html)
shows per-floor trends, but obscures the difference in amounts of observations.

In [ ]:
(p9.ggplot()
    + p9.geom_density(data=(mn_radon.loc[mn_radon['floor']==0]), mapping=p9.aes(x='log_radon'), color='darkblue')
    + p9.geom_density(data=(mn_radon.loc[mn_radon['floor']==1]), mapping=p9.aes(x='log_radon'), color='purple')
    + p9.xlab("log radon levels") + p9.ylab("")
    + p9.theme(figure_size=(6,4))
)

The [plotnine geom_point](https://plotnine.readthedocs.io/en/stable/generated/plotnine.geoms.geom_point.html) plots two variables as (x, y) points.
A scatterplot of points (`floor`, `log_radon`) will only have 2 distinct x-axis values:  0 and 1.
The [plotnine geom_jitter](https://plotnine.readthedocs.io/en/stable/generated/plotnine.geoms.geom_jitter.html) adds jitter to the (x, y) points,
which reduces the amount of overplotting and therefore allows for a better visualization of the amount of data being plotted.
Therefore we use the latter to visualize the differences between the the radon measurements by floor.

In [ ]:
plot_radon_floor = (p9.ggplot(data=mn_radon, mapping=p9.aes(x='floor', y='log_radon')) 
    + p9.geom_jitter(width=0.1, alpha=0.5, fill='orange', color='darkred')
    + p9.scale_x_continuous(breaks=range(0,2), minor_breaks=[])
    + p9.ggtitle("Radon measurements by floor")
    + p9.theme(figure_size=(4,4))
)
plot_radon_floor

**County-level information**

At the county level we have many home radon measurements from the relatively few counties with metropolitan areas, and very few home radon measurements from the rest.

In [ ]:
(p9.ggplot(data=mn_counties)
    + p9.geom_histogram(mapping=p9.aes(x='obs'), bins=40)
    + p9.geom_text(data=mn_counties[mn_counties['obs'] > 25],
                   mapping=p9.aes(x='obs', y=2, label='county'),
                   size=8, nudge_x=4, nudge_y=0.1)
    + p9.xlab("observations per county") + p9.ylab("")
    + p9.theme(figure_size=(12,4))
)

We use the [plotnine.geom_boxplot](https://plotnine.readthedocs.io/en/stable/generated/plotnine.geoms.geom_boxplot.html#plotnine.geoms.geom_boxplot)
to generate a [box_and_whiskers plot](https://en.wikipedia.org/wiki/Box_plot) for each set of per-county radon measurements.  The width of the box is proportional to the square root of the number of observations.
As the counties are ordered by number of observations, the width of the boxes increases from left to right.
The box encloses the central 25% - 75% quantiles; this is the 
[interquartile range (IQR)](https://en.wikipedia.org/wiki/Interquartile_range).
The the whiskers extend to the values which are a distance of 1.5 the IQR, and values beyond that are plotted as points - these are the outliers.

In [ ]:
obs_asc = mn_counties.sort_values(by='obs').reset_index(drop=True)

boxplot_radon_obs = (p9.ggplot(data=mn_radon, mapping=p9.aes(x='county',y='log_radon'))
    + p9.geom_boxplot(width=2, varwidth=True, outlier_alpha=0.4)
    + p9.scale_x_discrete(limits=obs_asc['county'], expand=(0,3))
    + p9.ggtitle("Counties ordered by number of observations per county")
    + p9.ylab("range of home radon measurements")
)                     
# horizontal
boxplot_radon_obs + xlabels_90 + p9.theme(figure_size=(20,6))

Because the most home radon measurements were taken at the basement floor level and because most of the counties have relatively few home measurments taken, there are many counties where all measurements are from the basement floor.

In [ ]:
print(f'Number of counties: {mn_radon.county.nunique()}')
print(f'Counties with measurements from floor 0: {mn_radon[mn_radon["floor"]==0]["county"].nunique()}')
print(f'Counties with measurements from floor 1: {mn_radon[mn_radon["floor"]==1]["county"].nunique()}')

**Relationship between home radon and county-level soil uranium**

At the county-level, we have information on the soil uranium level.  We plot the number of observations by soil uranium.  The points on the x-axis line up with the histogram bars on the above plot, but instead of histogram bars, we have a series of points showing the different log_uranium levels.

In [ ]:
(p9.ggplot(data=mn_counties, mapping=p9.aes(x='obs', y='log_uranium'))
    + p9.geom_point(fill='orange', color='darkred')
    + p9.geom_text(data=mn_counties[mn_counties['obs']>25],
                   mapping=p9.aes(label='county'),
                   size=8, nudge_x=4, nudge_y=0.1)
    + p9.xlab("observations per county") + p9.ylab("county soil log_uranium")
    + p9.theme(figure_size=(12,4))
)

We plot the relationship between soil uranium level and the home radon measurement.
We use plotnine's [facet_grid](https://plotnine.readthedocs.io/en/stable/generated/plotnine.facets.facet_grid.html#plotnine.facets.facet_gridplot) to get side-by-side per-floor plots.
Because the soil uranium level measurement is the same for all homes in a county, for counties with many houses, i.e., metropolitan areas, the plot shows distinct vertical bands.

In [ ]:
(p9.ggplot(data=mn_radon, mapping=p9.aes(x='log_uranium', y='log_radon'))
     + p9.geom_point(alpha=0.9, fill='orange', color='darkred')
     + p9.facet_grid(facets='~ floor', labeller='label_both')
     + p9.xlab("county-level soil log_uranium") + p9.ylab("home log_radon") 
     + p9.theme(figure_size=(12,4))
)

To see whether or not the soil uranium level might be a good predictor of the home radon measurements,
we can repeat the above boxplot, this time ordering the counties on the x-axis by
the per-county activity levels, ordered by uranium level per county, descending.


Given the sparse data, the resulting plot is inconclusive.

In [ ]:
uranium_desc = mn_counties.sort_values(by='log_uranium', ascending=False).reset_index()

(p9.ggplot(data=mn_radon, mapping=p9.aes(x='county',y='log_radon'))
    + p9.geom_boxplot(width=2, varwidth=True, outlier_alpha=0.4)
    + p9.scale_x_discrete(limits=uranium_desc['county'], expand=(0,1))
    + p9.ggtitle("Counties ordered by soil uranium high (left) to low (right)")
    + p9.ylab("range of home radon measurements")
    + xlabels_90
    + p9.theme(figure_size=(20,6))
)

## Linear regression review (chapters 2 and 3 of Gelman and Hill)

> Linear regression is a method that summarizes how the average values of a numerical outcome variable vary over subpopulations defined by linear functions of predictors. ... Regression can be used to predict an outcome given a linear function of these predictors, and regression coefficients can be thought of as comparisons across predicted values or as comparisons among averages in the data. </br>_Gelman and Hill, chapter 3_

[Linear regression](https://en.wikipedia.org/wiki/Linear_regression#Formulation) models the relationship between a scalar response and one or more explanatory variables.

<img width="190" alt="Linear least squares example2" align="right" style="vertical-align:middle;margin:0px 50px"
     src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/53/Linear_least_squares_example2.png/190px-Linear_least_squares_example2.png">

In linear regression, the observations (red) are assumed to be the result of random deviations (green) from an underlying relationship (blue) between a dependent variable (y) and an independent variable (x).

A [simple linear regression](https://en.wikipedia.org/wiki/Simple_linear_regression) is a model which relates
a pair of sample points, an  _independent variable_ x, and a _dependent variable_ y.
The adjective simple refers to the fact that the outcome variable is related to a single predictor.
The model finds a linear function (a non-vertical straight line) that,
as accurately as possible, predicts the dependent variable values as a function of the independent variable.

*Geometry review:* a straight line is defined by its _intercept_ ($\alpha$), the offset from zero on the x-axis, and _slope_ ($\beta$), the multiplier applied to x, i.e.,  $\mathrm{y} = \alpha + \beta\, \mathrm{x}$.

**Linear regression:  two ways to write the model**

The goal of inference is to learn from incomplete or imperfect data.
In the simple linear regression model, the error term $\epsilon$ accounts for imperfect measurments of the data.

$
y_i = \alpha \, + {\beta}\,x_i + {\epsilon}_i
$
where the errors ${\epsilon}_i$ have independent normal distributions with mean $0$ and standard deviation $\sigma$.

An equivalent representation is

$
y_i ∼ \mathrm{N}(\alpha + \beta\,\mathrm{X}_i,\, \sigma^2), \ \ \ \mathrm{for}\ i=1, ..., n
$

The corresponding Stan statement is

```
y ~ normal(alpha + beta * x, sigma);
```

Stan provides [vectorized](https://mc-stan.org/docs/functions-reference/vectorization.html#vectorization) versions of all univariate probability distrions.
This statement is far more efficient than using a `for` loop over all $x_i$ and $y_i$ pairs.

**Simple linear regression model in Stan**

The simple linear regression with a single predictor and a slope and intercept coefficient and normally distributed noise is the first model discussed in the [Stan User's Guide Regression Models chapter](https://mc-stan.org/docs/stan-users-guide/linear-regression.html).

```
data {
  int<lower=0> N;
  vector[N] x;
  vector[N] y;
}
parameters {
  real alpha;
  real beta;
  real<lower=0> sigma;
}
model {
  y ~ normal(alpha + beta * x, sigma);
}
```

This model is the minimal possible model.
It consists of three named [program blocks](https://mc-stan.org/docs/reference-manual/overview-of-stans-program-blocks.html):
the data and parameters are declared in the respectively named data and parameters block;
the model block specifies the likelihood, i.e., the probability of the data given the model.
Because no priors are specified on the model parameters, they are given the default prior
distribution, which is uniform from  $-\infty$ to $+\infty$.

## Best Practice #2: start with a simple model

Starting from a simple model ensures that there is a good baseline
against which to measure performance.
Gelman and Hill start from a simple linear regression which models the relationship between
the log radon measurement and the floor on which this measurement was taken,
i.e., in the regression model the outcome "y" is the log radon level
and the predictor "x" is the floor on which the measurement was taken.

$
\mathrm{log\_radon}_i = \alpha \, + {\beta}\,\mathrm{floor}_i + {\epsilon}_i
$

Since the basement level is coded as $\mathrm{floor} = 0$,
the observed outcome $\mathrm{log\_radon}$ is simply the value of $\alpha$.

The **complete pooling** model pools all measurements from all counties.
Because this is a simple linear regression, we can plot it directly in plotnine by adding
[plotnine.geom.geom_smooth(method="lm")](https://plotnine.readthedocs.io/en/stable/generated/plotnine.geoms.geom_smooth.html#plotnine.geoms.geom_smooth)
to the earlier plot of radon measurements by floor.
The black line is the estimated mean and the the grey margins indicate the amount of variance.

In [ ]:
p1 = plot_radon_floor +  p9.geom_smooth(method='lm')
p1

### Simple linear regression in Stan

We can use the simple linear regression model, above, to fit the Minnesota radon dataset.
This model fits a single regression line to all 919 observations in the dataset.
It is a "complete pooling" model because it ignores county information and therefore
pools all information from all counties.

We create a [CmdStanModel](https://mc-stan.org/cmdstanpy/api.html#cmdstanmodel) object from the Stan program file
[radon_cp.stan](stan/radon_cp.stan).
The Stan model corresponds to the simple linear regression model above, except that:

* There are non-default priors on all model parameters.  See https://github.com/stan-dev/stan/wiki/Prior-Choice-Recommendations for guidance. Here we use weakly informative priors, `normal(0, 10)`.

* We add a `generated quantities` block which will be used to do prior predictive checking (next section).


*Note:*  We use file name conventions "cp" for "complete pooling", "np" for "no pooling" and "pp" for "partial pooling".  *(Another good practice:  use naming conventions).*

In [ ]:
complete_pooling_model = CmdStanModel(stan_file=os.path.join('stan', 'radon_cp.stan'))
print(complete_pooling_model.code())

We assemble a Python dictionary which contains the definitions of the data block variables.

In [ ]:
radon_data = {"N": len(mn_radon), "x": mn_radon.floor.astype(float), "y": mn_radon.log_radon}

We call the model's [sample](https://mc-stan.org/cmdstanpy/api.html#cmdstanpy.CmdStanModel.sample)
method which runs Stan's NUTS-HMC sampler.

In [ ]:
complete_pooling_fit = complete_pooling_model.sample(data=radon_data)

The `sample` method returns a [CmdStanMCMC](https://mc-stan.org/cmdstanpy/api.html#cmdstanmcmc) object which
provides methods to summarize and diagnose the model fit and accessor methods to access the entire sample or individual items.
Accessor functions allow the user
to access the sample in whatever data format is needed for further analysis.

- The sample can be treated as a collection of named, structured variables

    + methods [stan_variable](https://mc-stan.org/cmdstanpy/api.html#cmdstanpy.CmdStanMCMC.stan_variable)
    and [stan_variables](https://mc-stan.org/cmdstanpy/api.html#cmdstanpy.CmdStanMCMC.stan_variables) return a `numpy.ndarray` and
    a Python dictionary of `numpy.ndarray` objects, respectively, whose structure corresponds to the Stan variable.

    + method [draws_xr](https://mc-stan.org/cmdstanpy/api.html#cmdstanpy.CmdStanMCMC.draws_xr) returns an [xarray.Dataset](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.html) of all Stan variables.

- The sample can be extracted in tabular format, either as

    + method [draws](https://mc-stan.org/cmdstanpy/api.html#cmdstanpy.CmdStanMCMC.draws) returns a [numpy.ndarray](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html#numpy.ndarray) over all columns in the output CSV file.

    + method [draws_pd](https://mc-stan.org/cmdstanpy/api.html#cmdstanpy.CmdStanMCMC.stan_variable) returns a [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html#pandas.DataFrame) over all columns in the output CSV file.  The argument `vars` can be used to restrict this to specified variables or columns.
    
In this example, we used the `draws_pd` method to access the sample draws for variables `alpha`, `beta`, and `sigma`.   This flattens the per-chain draws into a single column of all draws across all chains.

In [ ]:
pool_pd = complete_pooling_fit.draws_pd(vars=['alpha', 'beta', 'sigma'])
print(f'sample draws shape:  {pool_pd.shape}')
pool_pd[:3]

In [ ]:
pool_stats = pool_pd.describe()
pool_stats.round(2)

Another, more computationally expensive way to get these summary statistics is to call the [CmdStanMCMC.summary](https://mc-stan.org/cmdstanpy/api.html#cmdstanpy.CmdStanMCMC.summary) method.
This returns a pandas.DataFrame of summary statistics for total joint log probability lp__ and all model variables,
plus diagnostic statistics on the
[effective sample size](https://mc-stan.org/docs/reference-manual/effective-sample-size.html)
and [R_hat](https://mc-stan.org/docs/reference-manual/notation-for-samples-chains-and-draws.html#potential-scale-reduction), the potential scale reduction factor.

In [ ]:
complete_pooling_fit.summary().round(2)[1:4]

To check that the Stan model's estimates are in line with the `geom_smooth(method='lm')`  results shown above,
we create the same plot from the fitted model.

To add a trend line to the plot of radon measurements by floor, we can use either

+ [plotnine.geoms.geom_line](https://plotnine.readthedocs.io/en/stable/generated/plotnine.geoms.geom_line.html#plotnine.geoms.geom_line)
which draws a line through a set of connected points
+ [plotnine.stats.stat_function](https://plotnine.readthedocs.io/en/stable/generated/plotnine.stats.stat_function.html#plotnine.stats.stat_function)
which superimposes a function on a plot.

To see how these work, we do both:  we use the `stats.stat_function` to draw the mean trend line.
To see the amount of variance in our estimate, we plot a random sample of draws from the posterior,
using the `geoms.geom_line` function.

The function for a line is $ \mathrm{y} = \alpha + \beta\,* \,\mathrm{x}$.
For this dataset, the x values are $\{ 0, 1 \}$, which means that

+ when x $= 0$, y $=$ `alpha`
+ when x $= 1$, y $=$ `alpha` + `beta`.

In [ ]:
# get a random sample of the draws
sz = 100
f1 = pool_pd.alpha + pool_pd.beta  # y coord at floor 1 
f0 = pd.Series(pool_pd.alpha.values)  # y coord at floor 0
ys = pd.concat([f0, f1], axis=1)
ys = ys.sample(sz).reset_index(drop=True)

# add sample regression lines to plot_radon_floor (from earlier section)
p2 = plot_radon_floor
for i in range(sz):
    p2 = p2 + p9.geom_line(data=ys.T, mapping=p9.aes(x=[0,1], y=ys.loc[i]), inherit_aes=False, color='grey', alpha=0.06)

# add central regression line
p2 = p2 + p9.stat_function(mapping=p9.aes(x=1),
    fun=lambda x: pool_stats.alpha['mean'] + pool_stats.beta['mean']*x,
    color='blue', size=1
)
p2

In order to display multiple plotnine plots in one Jupyter notebook cell, we can use [patchworklib](https://github.com/ponnhide/patchworklib),
following the examples in [patchworklib-examples](https://github.com/ponnhide/patchworklib-examples/blob/main/example_notebooks/subplot4plotnine.ipynb).

The plot on the left are the estimates from the Stan model, the plot on the right are the plotnine `geom_smooth(method='lm')`, i.e., simple linear regression.

In [ ]:
g2 = pw.load_ggplot(p2)
g2.savefig(quick=True)
g1 = pw.load_ggplot(p1)
g2.savefig(quick=True)
p12 = (g2 | g1)
p12.case.set_title("modeled regression in Stan vs. plotnine smoothing", pad=24, size=18)
p12.savefig(quick=True)

## Best Practice #3: posterior predictive check

From the [Stan User's Guide](https://mc-stan.org/docs/stan-users-guide/ppcs.html)

> Posterior predictive checks are a way of measuring whether a model does a good job of capturing relevant aspects of the data, such as means, standard deviations, and quantiles (Donald B. Rubin 1984; Andrew Gelman, Meng, and Stern 1996). Posterior predictive checking works by simulating new replicated data sets based on the fitted model parameters and then comparing statistics applied to the replicated data set with the same statistic applied to the original data set.

See: https://mc-stan.org/docs/stan-users-guide/simulating-from-the-posterior-predictive-distribution.html

plot y_rep:   probability of new observations of y (y-tilde) conditional on y.

In [ ]:
# get a random sample of the draws
sz = 40
y_rep = complete_pooling_fit.draws_pd(vars='y_rep')

# each column is a replicate of the data, using estimates of alpha, beta
y_rep_sample = y_rep.sample(sz).reset_index(drop=True).T

# plot actual distribution of the data against predicted new data
cp_ppc = p9.ggplot()
for i in range(sz):
    cp_ppc = cp_ppc + p9.geom_density(mapping=p9.aes(x=y_rep_sample[i]), color='lightblue', alpha=0.4)
cp_ppc = (cp_ppc 
          + p9.geom_density(data=mn_radon, mapping=p9.aes(x='log_radon'), color='darkblue', size=1.1)
          + p9.xlab("log radon levels") + p9.ylab("")
          + p9.theme(figure_size=(6,4))
         )
cp_ppc

## Alternative simple linear regression models

An alternative to the complete pooling model is fitting a model to each county, using just the data from that county.
We can do this in plotnine by adding faceting to the above plot.

As noted above, most of the measurements are taken at the basement level and
there are 25 counties without any measurements taken on floor 1.
For these counties, in the faceted plot the `geom_smooth` trend line is absent;
i.e., we cannot estimate the slope of the regression line.
For counties with measurements on both floors, in a few cases, e.g., Carlton county,
the regression line between floor 0 and 1 has a positive slope, which
a) goes against the estimate from the complete pooling model, 
and b) goes against what we know about how radon enters the home.

In [ ]:
(p9.ggplot(data=mn_radon, mapping=p9.aes('floor', 'log_radon'))
     + p9.geom_jitter(width=0.05)
     + p9.geom_smooth(method='lm')
     + p9.facet_wrap('county')
     + p9.ggtitle("Per-county regressions")
     + p9.scale_x_continuous(breaks=range(0,2), minor_breaks=[])
     + p9.scales.ylim(-3, 4)  # same limits as complete pooling
     + p9.theme(figure_size=(18,20))
)

Gelman and Hill chapter 12.3 introduces variant of this model which
estimates per-county intercepts `alpha`
but it a single slope coefficient `beta`
and also assumes the residual variance is the same within each group.

$
\mathrm{log\_radon}_i = \alpha_{j[i]} \, + {\beta}\,\mathrm{floor}_i + {\epsilon}_i
$
where $j = 1 \ldots 85$

This model says that we expect the effect of the floor on which the measurement was taken is the same across counties, but that the baseline measurement is different.
Coding this model in Stan requires the following modifications to the complete pooling model

* we introduce data variable `J`, the number of counties.
* we introduce data variable `county`, the array of per-house county ids.
* we change `alpha` from a scalar to a vector of length is `J`, the number of counties.
* we compute an intermediate variable `eta` so that we can continue to use the
vectorized form of the sampling statement.

This model is in file [radon_np.stan](stan/radon_np.stan).

In [ ]:
no_pooling_model = CmdStanModel(stan_file=os.path.join('stan', 'radon_np.stan'))
print(no_pooling_model.code())

We add the necessary inputs to the `radon_data` dict and run the model, which computes the individaul intercepts `alpha`.

In [ ]:
radon_data["J"] = 85
radon_data["county"] = mn_radon.county_code
no_pooling_fit = no_pooling_model.sample(data=radon_data)
no_pooling_fit.summary().round(2)[83:90]

In [ ]:
# get a random sample of the draws
sz = 40
y_rep_np = no_pooling_fit.draws_pd(vars='y_rep')

# each column is a replicate of the data, using estimates of alpha, beta
y_rep_np_sample = y_rep_np.sample(sz).reset_index(drop=True).T

# plot actual distribution of the data against predicted new data
np_ppc = p9.ggplot()
for i in range(sz):
    np_ppc = np_ppc + p9.geom_density(mapping=p9.aes(x=y_rep_np_sample[i]), color='lightblue', alpha=0.4)
np_ppc = (np_ppc 
          + p9.geom_density(data=mn_radon, mapping=p9.aes(x='log_radon'), color='darkblue', size=1.1)
          + p9.xlab("log radon levels") + p9.ylab("")
          + p9.theme(figure_size=(6,4))
         )
np_ppc

In [ ]:
g3 = pw.load_ggplot(cp_ppc)
g3.savefig(quick=True)
g4 = pw.load_ggplot(np_ppc)
g4.savefig(quick=True)
p34 = (g3 | g4)
p34.case.set_title("complete pooling PPC vs. no pooling PPC", pad=24, size=18)
p34.savefig(quick=True)

To see the difference in the estimates of the intercept term `alpha` for these two models
we plot the no pooling model's per-county estimates of alpha and
overlay it with the complete pooling model estimate.
To show the mean and variance at 1 standard deviation, we use numpy's `quantile` function to get the upper and lower bounds.

In [ ]:
complete_pool_alpha_mean = complete_pooling_fit.stan_variable('alpha').mean()

no_pool_alpha = no_pooling_fit.stan_variable('alpha')
no_pool_alpha_mean = np.mean(no_pool_alpha, axis=0)  # axis=0 uses all rows, i.e., per-column mean
no_pool_alpha_lower = np.quantile(no_pool_alpha, 0.16, axis=0)
no_pool_alpha_upper = np.quantile(no_pool_alpha, 0.84, axis=0)
no_pool_alpha_pd = pd.DataFrame(data={
    "mean": no_pool_alpha_mean,
    "upper": no_pool_alpha_upper, 
    "lower": no_pool_alpha_lower, 
    "county":mn_counties['county']
})
no_pool_alpha_pd[:3]

We plot the estimates of `alpha` for all counties, sorted by number of observations, descending.
The orange horizontal line is the mean estimate of `alpha` from the complete pooling model.
The complete pooling model ignores the county level variance.  The no-pooling model overstates the variance between counties, especially for counties with only a few observations. The no-pooling estimates sometimes line up with the complete-pooling estimate, but not for the some of the counties with the most observations.

In [ ]:
# get sort order
pop_desc = mn_counties.sort_values(by='obs', ascending=False).reset_index()

# visualize
p_no_pool = (p9.ggplot(data=no_pool_alpha_pd)
 # Range strip
 + p9.geom_segment(
     mapping=p9.aes(x='county', xend='county', y='lower', yend='upper'),
     size=1.4, color='darkblue', alpha=0.5,
 )
 + p9.geom_point(mapping=p9.aes(x='county', y='mean'))
 + p9.geom_hline(yintercept=complete_pool_alpha_mean, color='darkorange', size=1.5)
 + p9.scale_x_discrete(limits=pop_desc['county'])
 + p9.ggtitle("No pooling model estimates for alpha (basement log_radon level)")
 + p9.xlab("ordered by observations per county, desc") + p9.ylab("central 67% interval")
 + xlabels_90
 + p9.theme(figure_size=(20,6)) 
)
p_no_pool

To see how the no-pooling estimate lines up with the raw data, we add the boxplot from the previous section to this plot.
The fewer the number of observations, the greater the variance in the estimates.
The value of the intercept corresponds to the average value of basement-level measurements in the data whereas the boxplot center line is the median value of the observations for that county.

In [ ]:
(p_no_pool
 + p9.geom_boxplot(data=mn_radon, mapping=p9.aes(x='county',y='log_radon'), color='orange', alpha=0.4, outlier_alpha=0.3)
)

### Simple linear regression in Stan:  adding predictors

We add the county-level soil log uranium measurements to the no-pooling model.
This model is in file [radon_np_uranium.stan](stan/radon_np_uranium.stan).

In [ ]:
two_predictors_model = CmdStanModel(stan_file=os.path.join('stan', 'radon_np_uranium.stan'))
print(two_predictors_model.code())

In [ ]:
radon_data["u"] = mn_radon.log_uranium

In [ ]:
two_predictors_fit = two_predictors_model.sample(data=radon_data)
two_predictors_fit.summary().loc['beta[2]'].round(2)

Although the mean estimate of the regression coefficient for the soil log uranium level is positive, it has extremely high variance and the central 90% of the probability distribution includes 0.
Plot this.

In [ ]:
# plot distribution
# get a random sample of the draws
sz = 40
y_rep_u = two_predictors_fit.draws_pd(vars='y_rep')

# each column is a replicate of the data, using estimates of alpha, beta
y_rep_u_sample = y_rep_u.sample(sz).reset_index(drop=True).T

# plot actual distribution of the data against predicted new data
u_ppc = p9.ggplot()
for i in range(sz):
    u_ppc = u_ppc + p9.geom_density(mapping=p9.aes(x=y_rep_u_sample[i]), color='lightblue', alpha=0.4)
u_ppc = (u_ppc 
          + p9.geom_density(data=mn_radon, mapping=p9.aes(x='log_radon'), color='darkblue', size=1.1)
          + p9.xlab("log radon levels") + p9.ylab("")
          + p9.theme(figure_size=(6,4))
         )
u_ppc


In [ ]:
# plot per-county range of estimates of beta

## Multilevel models

Multilevel models are extensions of regression which can model
dependency structures in the data.
For example, when data are nested in a hierarchy, a multi-level model
lets us model all levels of the hierarchy.
For this dataset, houses are nested within counties.
An ordinary regression can either model all counties as being identical, the complete pooling model,
or all counties as being different, the no-pooling model.

A multi-level regression accounts for the observed variation across counties
by modeling the counties as being drawn from
a common distribution, whose parameters are estimated jointly by the model.
This allows for *partial pooling* of information.
When there is very little variation across counties, the multi-level model approaches the complete-pooling model,
at the other extreme, when the amount of variation is very large, it approaches the no-pooling model

A simple linear regression model with a single predictor
estimates two parameters:  the intercept and slope of the regression line.
In a multilevel model, either or both of these parameters can be modeled, as illustrated by Figure 11.1 from Gelman and Hill.
As the number of regression predictors increases, the modeling choices increase.
![multilevel models](img/multilevel_models_fig_11_1.png)

###  Varying intercept models

Like the no-pooling model, this model has a vector of per-county intercept terms $\alpha$,
but in the multilevel model, we place a distribution on the intercept term

$
\alpha_j \sim \mathrm{N}(\mu_\alpha,\, {\sigma_\alpha}^2),\ \ \ \ \mathrm{for}\ \ j\, = 1, \ldots, \mathrm{J}
$

Putting this together with the individual intercept model above, the partial-pooling model is

$
y_i = \alpha_{j[i]} \, + {\beta}\,x_i + {\epsilon}_i\newline
\alpha_j \sim \mathrm{N}(\mu_\alpha,\, {\sigma_\alpha}^2),\ \ \ \ \mathrm{for}\ \ j\, = 1, \ldots, \mathrm{J}
$

All parameters are also estimated jointly by the model.
This model provides _partial pooling_ of information;
it pulls the estimates of $\alpha_j$ towards the mean level $\mu_\alpha$, to a greater or lesser degree.
Partial pooling is a _soft constraint_ whose effect depends on the amount of group-level variance $\sigma_\alpha$.
As the variance increases, the amound of pooling decreases so that when
$\sigma_\alpha \rightarrow \inf$ there is no pooling; and when $\sigma_\alpha \rightarrow 0$ there is complete-pooling.

The varying intercept model is in file [stan/radon_pp_alpha.stan](stan/radon_pp_alpha.stan).

In [ ]:
partial_pooling_alpha_model = CmdStanModel(stan_file=os.path.join('stan', 'radon_pp_alpha.stan'))
print(partial_pooling_alpha_model.code())

In [ ]:
partial_pooling_alpha_fit = partial_pooling_alpha_model.sample(data=radon_data)
partial_pooling_alpha_fit.summary().round(2)[:10]

In [ ]:
complete_pool_alpha = np.mean(complete_pooling_fit.stan_variable('alpha'))
complete_pool_alpha_lower = np.quantile(complete_pooling_fit.stan_variable('alpha'), .16, axis=0)
complete_pool_alpha_upper = np.quantile(complete_pooling_fit.stan_variable('alpha'), .84, axis=0)

part_pool_mu_alpha = partial_pooling_alpha_fit.stan_variable('mu_alpha').mean()
part_pool_alpha = partial_pooling_alpha_fit.stan_variable('alpha')
part_pool_alpha_mean = np.mean(part_pool_alpha, axis=0)
part_pool_alpha_lower = np.quantile(part_pool_alpha, 0.16, axis=0)
part_pool_alpha_upper = np.quantile(part_pool_alpha, 0.84, axis=0)
part_pool_alpha_pd = pd.DataFrame(
    data={
        "mean": part_pool_alpha_mean,
        "upper": part_pool_alpha_upper, 
        "lower": part_pool_alpha_lower, 
        "county":mn_counties['county']
    }
)
part_pool_alpha_pd[:3]

We plot the per-county estimates of `alpha` just as we did above, and we keep the y-axis on the same scale
as for the no-pooling model; which were in range $(0, 3.5)$ (roughly).   This shows how the hierarchical pools information
among the intercept terms and helps shrink the variance of the estimates.

In [ ]:
# visualize
p_partial_pool_intercept = (p9.ggplot(data=part_pool_alpha_pd)
 # Range strip
 + p9.geom_segment(
     mapping=p9.aes(x='county', xend='county', y='lower', yend='upper'),
     size=1.7, color='blue', alpha=0.7,
 )
 + p9.geom_point(mapping=p9.aes(x='county', y='mean'))
 + p9.geom_hline(yintercept=part_pool_mu_alpha, color='darkblue', size=1)
 + p9.scale_x_discrete(limits=pop_desc['county']) + p9.scales.ylim(0,3.5)
 + p9.ggtitle("multilevel varying intercept model estimates for alpha (basement log_radon level)")
 + p9.xlab("ordered by observations per county, desc") + p9.ylab("central 67% interval")
 + xlabels_90
 + p9.theme(figure_size=(20,6)) 
)
p_partial_pool_intercept

To compare the no-pooling and partial pooling model estimates of `alpha` directly, we overlay the above plot with the estimated from the no-pooling model in orange.

In [ ]:
(p_partial_pool_intercept
    + p9.geom_segment(data=no_pool_alpha_pd,
         mapping=p9.aes(x='county', xend='county', y='lower', yend='upper'),
         size=1.4, color='orange', alpha=0.6,
     )
     + p9.geom_hline(yintercept=complete_pool_alpha_mean, color='darkorange', size=1)
)

In [ ]:
(p_partial_pool_intercept
 + p9.geom_boxplot(data=mn_radon, mapping=p9.aes(x='county',y='log_radon'), color='orange', alpha=0.4, outlier_alpha=0.3)
)

In [ ]:
# plot distribution
# get a random sample of the draws
sz = 40
y_rep_pp = partial_pooling_alpha_fit.draws_pd(vars='y_rep')

# each column is a replicate of the data, using estimates of alpha, beta
y_rep_pp_sample = y_rep_pp.sample(sz).reset_index(drop=True).T

# plot actual distribution of the data against predicted new data
pp_ppc = p9.ggplot()
for i in range(sz):
    pp_ppc = pp_ppc + p9.geom_density(mapping=p9.aes(x=y_rep_pp_sample[i]), color='lightblue', alpha=0.4)
pp_ppc = (pp_ppc 
          + p9.geom_density(data=mn_radon, mapping=p9.aes(x='log_radon'), color='darkblue', size=1.1)
          + p9.xlab("log radon levels") + p9.ylab("")
          + p9.theme(figure_size=(6,4))
         )
pp_ppc


### Adding predictors

We can add the county-level soil log uranium measure to the partial pooling model along with the floor, as was we did
for the complete pooling model.
This model is in file [radon_pp_uranium.stan](stan/radon_pp_uranium.stan).

In [ ]:
radon_pp_alpha_uranium = CmdStanModel(stan_file=os.path.join('stan', 'radon_pp_uranium.stan'))
print(radon_pp_alpha_uranium.code())

In [ ]:
radon_pp_alpha_uranium_fit = radon_pp_alpha_uranium.sample(data=radon_data)

In [ ]:
radon_pp_alpha_uranium_fit.summary().round(2)

In this model, there is a single intercept $\alpha$ and a  modeled predictor coefficients $\beta_j$.

$
\beta_j \sim \mathrm{N}(\mu_\beta,\, {\sigma_\beta}^2),\ \ \ \ \mathrm{for}\ \ j\, = 1, \ldots, \mathrm{J}
$

Putting this together with the individual intercept model above, the partial-pooling model is

$
y_i = \alpha \, + {\beta_{j[i]}}\,x_i + {\epsilon}_i\newline
\beta \sim \mathrm{N}(\mu_\beta,\, {\sigma_\beta}^2),\ \ \ \ \mathrm{for}\ \ j\, = 1, \ldots, \mathrm{J}
$

In [ ]:
partial_pooling_beta_model = CmdStanModel(stan_file=os.path.join('stan', 'radon_partial_pool_beta.stan'))
print(partial_pooling_beta_model.code())

In [ ]:
partial_pooling_beta_fit = partial_pooling_beta_model.sample(data=radon_data)
partial_pooling_beta_fit.summary().round(2)

In [ ]:
complete_pool_beta = complete_pooling_fit.stan_variable('beta')
complete_pool_beta_mean = np.mean(complete_pool_beta)
complete_pool_beta_lower = np.quantile(complete_pool_beta, 0.16)
complete_pool_beta_upper = np.quantile(complete_pool_beta, 0.84)

In [ ]:
part_pool_mu_beta = partial_pooling_beta_fit.stan_variable('mu_beta').mean()
part_pool_beta = partial_pooling_beta_fit.stan_variable('beta')
part_pool_beta_mean = np.mean(part_pool_beta, axis=0)
part_pool_beta_lower = np.quantile(part_pool_beta, 0.16, axis=0)
part_pool_beta_upper = np.quantile(part_pool_beta, 0.84, axis=0)
part_pool_beta_pd = pd.DataFrame(
    data={
        "mean": part_pool_beta_mean,
        "upper": part_pool_beta_upper, 
        "lower": part_pool_beta_lower, 
        "county":mn_counties['county']
    }
)
part_pool_beta_pd[:3]

In [ ]:
# visualize
p_partial_pool_slope = (p9.ggplot(data=part_pool_beta_pd)
 # Range strip
 + p9.geom_segment(
     mapping=p9.aes(x='county', xend='county', y='lower', yend='upper'),
     size=1.4, color='darkblue', alpha=0.5,
 )
 + p9.geom_point(mapping=p9.aes(x='county', y='mean'))
 + p9.geom_hline(yintercept=part_pool_mu_beta, color='darkblue', size=1)
 + p9.geom_hline(yintercept=complete_pool_beta_mean, color='darkorange', size=1)
 + p9.geom_hline(yintercept=complete_pool_beta_upper, color='darkorange', size=0.7, linetype='dashed')
 + p9.geom_hline(yintercept=complete_pool_beta_lower, color='darkorange', size=0.7, linetype='dashed')
 + p9.scale_x_discrete(limits=pop_desc['county'])
 + p9.ggtitle("multilevel varying slope model estimates for beta (floor effect)")
 + p9.xlab("ordered by observations per county, desc") + p9.ylab("central 67% interval")
 + xlabels_90
 + p9.theme(figure_size=(20,6)) 
)
p_partial_pool_slope

This version of the model returns highly uncertain estimates for the floor effect.
From what we know about radon, we don't expect the floor effect to vary dramatically from county to county;
this result confirms that.

### Complex multilevel models

The full generality of multilevel modelling is introduced in Gelman and Hill chapter 13
"Multilevel linear models: varying slopes, non-nested models, and other complexities".
This introduces models in which slopes, intercepts, and interaction terms can all vary by group,
as well as non-nested models, where the dataset can be structured into groups in more than one way.
Examples of the latter are cross-cutting classifications such as demographic and geographic factors.

The simplest possible statement of the varying intercept, varying slope model for
the radon model with a single predictor models the both the intercepts and slopes separately,
without consideration to interaction between the per-county slope and intercept.

$
y_i = \alpha_{j[i]} \, + {\beta_{j[i]}}\,x_i + {\epsilon}_i\newline
\alpha_j \sim \mathrm{N}(\mu_\alpha,\, {\sigma_\alpha}^2),\newline
\beta_j \sim \mathrm{N}(\mu_\beta,\, {\sigma_\beta}^2),\ \ \ \ \mathrm{for}\ \ j\, = 1, \ldots, \mathrm{J}
$

Inasmuch as we don't believe that per-county estimates of the floor effect are warrented, we won't implement this model here.
For more information on the complexities of this model and how to code it efficiently in Stan, see the Stan User's guide,
section [Multivariate priors for hierarchical models](https://mc-stan.org/docs/stan-users-guide/multivariate-hierarchical-priors.html)

## Discussion

## Appendix A:  efficient Stan models

The models in this case study are all simple linear regression models, characterized by

+ a vector of observations of length `N`
+ a matrix of predictors of size `[N, M]`
+ the scalar intercept parameter
+ a vector of length `M` regression coefficients
+ variance parameter

These can be rewritten to use the more efficient [`normal_id_glm` distribution](https://mc-stan.org/docs/functions-reference/normal-id-glm.html) instead of the `normal` distribution.

A regression with multiple predictors relates the dependent variable `y` to a vector of independent variables `xs`.
The simple linear regression model generalizes to

$y_i = \alpha \, + B\,X_i + {\epsilon}_i$


where B is a vector of length M, and X is a matrix of size (N, M).

This model can be coded efficiently in Stan using the 
[normal_id_glm distribution](https://mc-stan.org/docs/functions-reference/normal-id-glm.html)
which is optimized for a [Generalised Linear Models (GLM)](https://en.wikipedia.org/wiki/Generalized_linear_model)
with a normal likelihood and identify link function, and is named accordingly.
The `normal_id_glm` distribution takes as arguments

+ a vector of observations of length `N`
+ a matrix of predictors of size `[N, M]`
+ the intercept parameter, either a scalar or a vector of length `M`
+ a vector of length `M` regression coefficients
+ variance parameter

```
/* normal linear regression model - single intercept */
data {
  int<lower=0> N;  // number of observations
  int<lower=1> M;  // number of predictors
  matrix[N, M] xs;
  vector[N] y;
}
parameters {
  real alpha;
  vector beta[M];
  real<lower=0> sigma;
}
model {
  y ~ normal_glm_id(xs, alpha, beta, sigma);
  sigma ~ normal(0, 10);   // override default prior uniform(-inf, +inf)
}

```

```
/* normal linear regression model - varying intercept */
data {
  int<lower=0> N;  // number of observations
  int<lower=1> M;  // number of predictors
  matrix[N, M] xs;
  vector[N] y;
}
parameters {
  vector alpha[N];
  vector beta[M];
  real<lower=0> sigma;
}
model {
  y ~ normal_glm_id(xs, alpha, beta, sigma);
  sigma ~ normal(0, 10);   // override default prior uniform(-inf, +inf)
}
```




## Appendix B:  Data preparation using pandas.

The steps required to convert the CSV files from the Gelman and Hill ARM website
into the data structures used in this analysis:

+ Merge the county-level soil uranium level measurment into the home radon data.
+ Put both outcome and predictors on the log scale, following Gelman and Hill, chapter 4, section 12.
+ Restrict the dataset to Minnesota.
+ Aggregate county-level information

Pandas objects contain structured arrays which are labeled by
[Index objects](https://pandas.pydata.org/docs/reference/indexing.html).
A [Series](https://pandas.pydata.org/docs/reference/series.html) object 
manages 1-D arrays and a [DataFrame](https://pandas.pydata.org/docs/reference/frame.html)
is a 2-D size-mutable, table, allowing for heterogenous columns.
These index labels are used to perform database-like select, join, and group-by operations.
However, sometimes we need to access just the data, not the index labels.
Most data is backed by an NumPy [ndarray](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html#numpy.ndarray).
The [array](https://pandas.pydata.org/docs/reference/api/pandas.Series.array.html) property
returns the underlying numpy.ndarray of the Index or Series object.

**Extract relevant columns from CSV as pandas DataFrame**

We leverage the [pandas.read_csv](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) function
to extract the information we need from the raw CSV files with the following non-default arguments

* parameter `usecols` allows us to extract just the relevant columns for this analysis.
* parameter `skipinitialspace` strips out initial whitespace from the data.

Once instantiated, we call the [convert_dtypes method](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.convert_dtypes.html) on the newly instantiated DataFrame so that we can do merge and join operations on all columns.

In [ ]:
df_radon = pd.read_csv(os.path.join('data','raw_radon.csv'),
    usecols=['state', 'stfips', 'floor', 'activity', 'county', 'cntyfips'],
    skipinitialspace=True,    # CSV file has spaces after delimiter, ignore them
).convert_dtypes()
print(f'Total records: {len(df_radon)}')
df_radon[:3]

In [ ]:
df_uranium = pd.read_csv(os.path.join('data','raw_uranium.csv'),
                        usecols=['stfips', 'ctfips', 'st', 'cty', 'Uppm'],
                        skipinitialspace=True,
                        ).drop_duplicates().convert_dtypes()
df_uranium[:3]

**Combine datasets**

[FIPS code](https://transition.fcc.gov/oet/info/maps/census/fips/fips.txt) are numbers which uniquely identify geographic areas. Both datasets have codes for the state and county ids, but these need to be combined to get a national-level county FIPS code.   In order to do a database-style join on the two tables, we need to

1. add acommon key to both tables
2. add the county-level soil uranium levels to the radon survey via the [DataFrame.merge](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html) method.

In [ ]:
df_radon['fips'] = df_radon.stfips*1000 + df_radon.cntyfips
df_uranium['fips'] = df_uranium.stfips*1000 + df_uranium.ctfips

df_radon = df_radon.merge(df_uranium[['fips', 'Uppm']], on='fips')
df_radon[:3]

**Put data on log scale**

Following Gelman and Hill chapter 4, section 4, we work with data on the log scale,
for two reasons

+ the outcome variable log_radon is always positive.
+ it provides modeling flexibility.

We know from geology that both radon measurements and soil uranium levels are always greater than zero,
however a few radon measurements in the EPA dataset are 0.
In order to be able to work with these measurements on the log scale, we replace 0 with 0.1,
which corresponds to a low radon level (following Gelman and Hill).

In [ ]:
df_radon['radon'] = df_radon.activity.apply(lambda x: x if x > 0. else 0.1)
df_radon['log_radon'] = np.log(df_radon['radon'])

df_radon['uranium'] = df_radon.Uppm.apply(lambda x: x if x > 0. else 0.1)
df_radon['log_uranium'] = np.log(df_radon['uranium'])
df_radon[:3]

**Cleanup**

Remove the columns which contain redundant information.

In [ ]:
df_radon.drop(columns=['stfips', 'activity', 'cntyfips', 'Uppm', 'fips', 'radon', 'uranium'], inplace=True)

**Restrict dataset to Minnesota**

In order to work with just the data from Minnesota, we use a 
use a conditional expression to [filter specific rows of a dataframe](https://pandas.pydata.org/docs/getting_started/intro_tutorials/03_subset_data.html#how-do-i-filter-specific-rows-from-a-dataframe), combined with operation [reset_index(drop=True)](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reset_index.html?highlight=reset_index#pandas.DataFrame.reset_index) so that the rows are indexed starting from 0.

In [ ]:
mn_radon = df_radon[df_radon['state']=='MN'].reset_index(drop=True)
mn_radon[:3]

**Add 1-based index code for MN counties**

The data inputs to a Stan model include a 1-based county index for each observation.
In order to do this we need to first get a sorted list of county names and then convert these to category code.

In [ ]:
counties = mn_radon.county.unique()
mn_radon['county_code'] = mn_radon.county.astype(
    pd.api.types.CategoricalDtype(categories=counties)
).cat.codes + 1  ## Stan indexes from 1
mn_radon[:5]

**Create auxiliary dataset of per-county information**

County-level information includes the number of observations taken in that county as well as the soil uranium level.
In order to easily visualize this information using plotnine, we create a secondary pandas.Dataframe object with per-county leel information.

The [value_counts](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.value_counts.html) method returns a Series containing counts of unique values,
We add these to the county-level dataframe.

In [ ]:
mn_uranium = mn_radon.iloc[mn_radon.county.drop_duplicates().index].reset_index(drop=True)
mn_uranium = mn_uranium.drop(columns=['floor', 'log_radon'])

**Save as CSV files**

These files are already part of this notebook, therefore calls to the  [pandsa.to_csv](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html?highlight=to_csv#pandas.DataFrame.to_csv) method have been commented out.

In [ ]:
mn_uranium['obs'] = mn_radon.value_counts(subset='county', sort=False).array
mn_uranium

In [ ]:
# uncomment as needed
mn_radon.to_csv(r'mn_radon.csv', index=False)
mn_uranium.to_csv(r'mn_uranium.csv', index=False)